In [1]:
import auxiliary_tools
from tqdm import tqdm 
from geopy.distance import geodesic

In [2]:
import array
import random
import numpy as np
import json
import pickle
import numpy
from math import sqrt
from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools

In [3]:
def generate_individual(creator, route_requests, rand_dist_min, rand_dist_max):
    individual = []
    for request in route_requests:
        
        rand_distance = random.randint(rand_dist_min, rand_dist_max)/1000
        rand_angle = random.randint(1, 360)
        
        gene = geodesic(kilometers=rand_distance).destination(request, rand_angle)[:2]
        
        individual.append(gene)
    individual = np.array(individual)
    return creator.individual(individual)

In [4]:
def mutation(individual, mutation_probability, rand_dist_min, rand_dist_max):
    mutated_individual = []
    for gene in individual:
        if random.random() < mutation_probability:
            rand_distance = random.randint(rand_dist_min, rand_dist_max)/1000
            rand_angle = random.randint(1, 360)
            
            mutated_gene = geodesic(kilometers=rand_distance).destination(gene, rand_angle)[:2]
            mutated_individual.append( mutated_gene )
        else:
            mutated_individual.append( gene )
    return  creator.individual(np.array(mutated_individual))

In [5]:
def crossover(individual_a, individual_b, crossover_probability):
    child_a = []
    child_b = []

    for i, (gene_a, gene_b) in enumerate(zip(individual_a, individual_b)):
        if random.random() < crossover_probability:
            child_a.append(gene_b)
            child_b.append(gene_a)
        else:
            child_a.append(gene_a)
            child_b.append(gene_b)

    return (creator.individual(np.array(child_a)), creator.individual(np.array(child_b)))

In [6]:
def client_fitness(route_requests, individual):
    c_fitness = []
    for i in range(len(route_requests)):
        request_r = route_requests[i]
        request_origin = [request_r[0], request_r[1]]
        vs_individual = individual[i]
        vs_destination = vs_individual
        c_fitness.append(auxiliary_tools.getGeoDistanceETA_OSRM(request_origin, vs_destination, 5003, 'walking'))
    fitness_value = np.sum([f[0] for f in c_fitness])
    return fitness_value

def operator_fitness(individual, penalty_const):
    ori_dest = [(first, second) for first, second in zip(individual, individual[1:])]
    penalty_sum = 0
    for pair in ori_dest:
        if max(pair[0] != pair[1]) == True:
            penalty_sum+=penalty_const
    o_fitness = []
    for od_r in ori_dest:
        o_fitness.append(auxiliary_tools.getGeoDistanceETA_OSRM(od_r[0], od_r[1], 5002, 'driving'))
        
    fitness_value = np.sum([f[0] for f in o_fitness]) + penalty_sum
    return fitness_value

def fitness(individual, route_requests, penalty_const):
    import time
#     start_time = time.time()
    from pexecute.thread import ThreadLoom
    loom = ThreadLoom(max_runner_cap=10)
    
    loom.add_function(client_fitness, [route_requests, individual], {})
    loom.add_function(operator_fitness, [individual, penalty_const], {})

    output = loom.execute()
    client_f = output[0]['output']
    operator_f = output[1]['output']
#     print("--- %s seconds ---" % round(time.time() - start_time))
    return client_f, operator_f

In [7]:
penalty_const = auxiliary_tools.getPenaltyConst(2)

route_requests = auxiliary_tools.loadPrep(1, 0)

crossover_probability = 0.8
mutation_probability = 0.5

rand_dist_min = 0
rand_dist_max = 500

population_size = 25
number_generations = 100

idx_evol = 8

# proposals:  123402
# requests 121427
# rides 46616


In [8]:
import time
start_time = time.time()

In [9]:
creator.create("min_fitness", base.Fitness, weights=(-1.0, -1.0))
creator.create("individual", list, fitness=creator.min_fitness)

toolbox = base.Toolbox()

toolbox.register("create_individual", generate_individual, creator, route_requests=route_requests,  rand_dist_min=rand_dist_min, rand_dist_max=rand_dist_max)
toolbox.register("initialize_population", tools.initRepeat, list, toolbox.create_individual)

toolbox.register("evaluate", fitness, route_requests=route_requests, penalty_const=penalty_const)
toolbox.register("crossover", crossover, crossover_probability=crossover_probability)
toolbox.register("mutate", mutation, mutation_probability=mutation_probability, rand_dist_min=rand_dist_min, rand_dist_max=rand_dist_min)

toolbox.register("select", tools.selNSGA2)

In [10]:
population = toolbox.initialize_population(n=population_size)

In [11]:
population_fitnesses = [toolbox.evaluate(individual) for individual in tqdm(population)]

100%|██████████| 25/25 [12:36<00:00, 30.25s/it]


In [12]:
for individual, fitness in zip(population, population_fitnesses):
    individual.fitness.values = fitness

In [13]:
list_generations_mean_fitness = []
list_generations_std_fitness = []
list_generations_min_fitness = []
list_generations_max_fitness = []

list_population_generations = []
list_best_individuo_generations = []
list_best_individuo_fitness_generations = []

In [14]:
current_generation = 0
while current_generation < number_generations:
    offspring = list(map(toolbox.clone, population))
    crossed_offspring = []

    # Crossing
    for child_1, child_2 in zip(offspring[::2], offspring[1::2]):
        child_a, child_b = toolbox.crossover(child_1, child_2)
        del child_a.fitness.values
        del child_b.fitness.values
        crossed_offspring.append(child_a)
        crossed_offspring.append(child_b)

    # Mutation
    for mu in range(0,len(crossed_offspring)):
        mutant = toolbox.mutate(crossed_offspring[mu])
        del mutant.fitness.values
        crossed_offspring[mu] = mutant

    # Fitness
    crossed_offspring_fitnesses = [toolbox.evaluate(individual) for individual in tqdm(crossed_offspring)]
    for individual, fitness in zip(crossed_offspring, crossed_offspring_fitnesses):
        individual.fitness.values = fitness

    new_population = population+crossed_offspring

    # Selection
    selected = toolbox.select(new_population, population_size)
    population_selected = list(map(toolbox.clone, selected))

    population[:] = population_selected

    print("---- STATISTICS GENERATION %s ----" % (current_generation))

    fits_client = [ind.fitness.values[0] for ind in population]
    fits_operator = [ind.fitness.values[1] for ind in population]

    length = len(population)
    mean_client = sum(fits_client) / length
    mean_operator= sum(fits_operator) / length
    sum2_client = sum(x*x for x in fits_client)
    sum2_operator = sum(x*x for x in fits_operator)
    std_client = abs(sum2_client / length - mean_client**2)**0.5
    std_operator = abs(sum2_operator / length - mean_operator**2)**0.5
    
    list_generations_mean_fitness.append((mean_client, mean_operator))
    list_generations_std_fitness.append((std_client, std_operator))
    list_generations_min_fitness.append((min(fits_client), min(fits_operator)))
    list_generations_max_fitness.append((max(fits_client), max(fits_operator)))
    
    list_population_generations.append(population)
    
    print("  Min %s %s" % (min(fits_client), min(fits_operator)))
    print("  Max %s %s" % (max(fits_client), max(fits_operator)))
    print("  Avg %s %s" % (mean_client, mean_operator))
    print("  Std %s %s" % (std_client, std_operator))
    
    current_generation+= 1
    
    best_ind = tools.selBest(population, 1)[0]
    
    list_best_individuo_generations.append(best_ind)
    list_best_individuo_fitness_generations.append(tuple(best_ind.fitness.values))
    
    print("Best individual is %s, %s" % (best_ind.fitness.values))
    
    data_ga = [list_generations_mean_fitness, list_generations_std_fitness,
              list_generations_min_fitness, list_generations_max_fitness,
              list_population_generations, list_best_individuo_generations,
              list_best_individuo_fitness_generations]
    
    pickle.dump(data_ga, open("data_ga_evol_"+str(idx_evol)+".pkl", "wb"))
print("--- %s seconds ---" % round(time.time() - start_time))

100%|██████████| 24/24 [15:20<00:00, 38.36s/it]


---- STATISTICS GENERATION 0 ----
  Min 776.6078 7535.108485964724
  Max 820.4495999999999 7643.833885964726
  Avg 795.6784399999998 7582.768861964722
  Std 10.757863163823348 24.672606419157574
Best individual is 776.6078, 7576.796585964725


100%|██████████| 24/24 [16:36<00:00, 41.52s/it]


---- STATISTICS GENERATION 1 ----
  Min 765.8934999999999 7529.549985964725
  Max 824.7535 7643.833885964726
  Avg 793.1993280000002 7574.717293964725
  Std 13.301671138566043 28.7246182408562
Best individual is 765.8934999999999, 7553.769685964725


100%|██████████| 24/24 [16:03<00:00, 40.15s/it]


...

100%|██████████| 24/24 [22:34<00:00, 56.45s/it]


---- STATISTICS GENERATION 60 ----
  Min 684.7592 7330.488785964724
  Max 700.3723 7353.860285964725
  Avg 689.9740480000002 7341.539557964724
  Std 4.044618053059862 7.738229124614808
Best individual is 684.7592, 7353.860285964725


100%|██████████| 24/24 [20:48<00:00, 52.03s/it]


---- STATISTICS GENERATION 61 ----
  Min 684.7592 7330.488785964724
  Max 697.187 7353.860285964725
  Avg 689.5277520000001 7339.869713964726
  Std 3.5071823432420652 7.535820635075905
Best individual is 684.7592, 7353.860285964725


KeyboardInterrupt: 

In [ ]:
import pickle

In [ ]:
best_ind = tools.selBest(population, 1)[0]

In [ ]:
pickle.dump(np.array(best_ind), open("best_individual"+str(idx_evol)+".pkl", "wb"))